In [1]:
import pandas as pd
import pyodbc

# -------------------------
# SQL Server connection
# -------------------------
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)

# -------------------------
# Client list
# -------------------------
x = [
    'AmulManagement','CholayilManagement'
]

# -------------------------
# Store summary info
# -------------------------
summary_data = []

# -------------------------
# Fetch data per client
# -------------------------
for client in x:
    query = f"""
        SELECT *,
                CONCAT(UserName,'-',LISStoreCode,'-',ProductCode,'-',FORMAT(TRY_CONVERT(date, [SubmittedDate&Time], 105), 'dd')) AS UserKey
        FROM {client}.PrimaryShelfStockReportExport
        WHERE MONTH(TRY_CONVERT(date, [SubmittedDate&Time], 105)) = 1 AND Year = '2026';
    """

    df = pd.read_sql(query, conn)

    # Clean column names
    df.columns = df.columns.str.strip()

    # Ensure UserKey exists
    if 'UserKey' not in df.columns or df['UserKey'].isna().all():
        df['UserKey'] = (
            df['UserName'].astype(str) + '-' +
            pd.to_datetime(df['Date'], errors='coerce')
              .dt.day.astype(str).str.zfill(2)
        )

    # Create dynamic DataFrame variable
    df_name = f"{client}_PrimarySelfStock"
    globals()[df_name] = df

    print(f"{df_name} created with {len(df)} rows")

    # Collect summary info
    summary_data.append({
        "ClientName": client,
        "No_of_Rows": len(df)
    })

# -------------------------
# Client-wise summary (ONLY ONCE)
# -------------------------
summary_df = pd.DataFrame(summary_data)
print("\nClient-wise Primary Self Stock Summary:")
print(summary_df)



C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_44980\281565640.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


AmulManagement_PrimarySelfStock created with 173631 rows


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_44980\281565640.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CholayilManagement_PrimarySelfStock created with 24605 rows

Client-wise Primary Self Stock Summary:
           ClientName  No_of_Rows
0      AmulManagement      173631
1  CholayilManagement       24605


In [2]:
import pandas as pd
Amul_PrimaryShelfStock = pd.read_csv(r"C:\Users\Administrator\LIS\Rahul\Primary-Self\Amul_PrimaryShelfStock.csv")

# Handle mixed date formats safely
Amul_PrimaryShelfStock["Submitted Date&Time"] = pd.to_datetime(
    Amul_PrimaryShelfStock["Submitted Date&Time"],
    format="mixed",
    dayfirst=True
)
# Create new column: UserName-Day
Amul_PrimaryShelfStock["UserName_Day"] = Amul_PrimaryShelfStock["Username"] + "-" + Amul_PrimaryShelfStock["LIS Store Code"] + "-" + Amul_PrimaryShelfStock["Product Code"]+ "-" + Amul_PrimaryShelfStock["Submitted Date&Time"].dt.strftime("%d")

Cholayil_PrimaryShelfStock = pd.read_csv(r"C:\Users\Administrator\LIS\Rahul\Primary-Self\Cholayil_PrimaryShelfStock.csv")

# Handle mixed date formats safely
Cholayil_PrimaryShelfStock["Submitted Date&Time"] = pd.to_datetime(
    Cholayil_PrimaryShelfStock["Submitted Date&Time"],
    format="mixed",
    dayfirst=True
)
# Create new column: UserName-Day
Cholayil_PrimaryShelfStock["UserName_Day"] = Cholayil_PrimaryShelfStock["Username"] + "-" + Cholayil_PrimaryShelfStock["LIS Store Code"] + "-" + Cholayil_PrimaryShelfStock["Product Code"]+ "-" + Cholayil_PrimaryShelfStock["Submitted Date&Time"].dt.strftime("%d")


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_44980\3743890238.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  Amul_PrimaryShelfStock = pd.read_csv(r"C:\Users\Administrator\LIS\Rahul\Primary-Self\Amul_PrimaryShelfStock.csv")


In [3]:
# Amul = pd.merge(
#     Amul_Store_Attendance,
#     AmulManagement_Store_Attendance, 
#     left_on="UserName_Day",
#     right_on="UserKey",
#     how="left"
# )

# print("Number of unmatched rows for Amul:", Amul['UserName_Day'].isna().sum())

# Cholayil = pd.merge(
#     Cholayil_Store_Attendance,
#     CholayilManagement_Store_Attendance,
#     left_on="UserName_Day",
#     right_on="UserKey",
#     how="left"
# )

# print("Number of unmatched rows for Cholayil:", Cholayil['UserName_Day'].isna().sum())


In [4]:
import pandas as pd
from datetime import datetime
import os

# -------------------------
# Current date
# -------------------------
today_str = datetime.now().strftime("%Y-%m-%d")

# -------------------------
# Client list (ONLY those loaded)
# -------------------------
clients = ['Amul', 'Cholayil']

# -------------------------
# Store summary
# -------------------------
unmatched_summary = []

# -------------------------
# Merge & calculate unmatched rows
# -------------------------
for client in clients:

    # CSV data
    left_df = globals().get(f"{client}_PrimaryShelfStock")

    # SQL data (⚠️ Self, not Shelf)
    right_df = globals().get(f"{client}Management_PrimarySelfStock")

    if left_df is None:
        print(f"Skipping {client}: CSV DataFrame missing")
        continue

    if right_df is None:
        print(f"Skipping {client}: SQL DataFrame missing")
        continue

    # -------------------------
    # Merge
    # -------------------------
    merged_df = pd.merge(
        left_df,
        right_df,
        left_on="UserName_Day",
        right_on="UserKey",
        how="left"
    )

    # -------------------------
    # Unmatched count
    # -------------------------
    unmatched_count = merged_df["UserKey"].isna().sum()

    unmatched_summary.append({
        "Date": today_str,
        "ClientName": client,
        "Portal_Records": len(left_df),
        "SQL_Records": len(right_df),
        "Unmatched_Rows": unmatched_count
    })

    # Store merged result
    globals()[f"{client}_Merged"] = merged_df

    print(f"{client} → Unmatched Rows: {unmatched_count}")

# -------------------------
# Final Summary
# -------------------------
unmatched_summary_df = pd.DataFrame(unmatched_summary)

print("\nClient-wise Unmatched Summary:")
print(unmatched_summary_df)

# -------------------------
# Save CSV
# -------------------------
output_path = "PrimaryShelfStock_Unmatched_Summary.csv"

if os.path.exists(output_path):
    unmatched_summary_df.to_csv(output_path, mode="a", header=False, index=False)
else:
    unmatched_summary_df.to_csv(output_path, index=False)

print(f"\nSummary CSV updated successfully at: {output_path}")


Amul → Unmatched Rows: 0
Cholayil → Unmatched Rows: 0

Client-wise Unmatched Summary:
         Date ClientName  Portal_Records  SQL_Records  Unmatched_Rows
0  2026-01-07       Amul          173631       173631               0
1  2026-01-07   Cholayil           24605        24605               0

Summary CSV updated successfully at: PrimaryShelfStock_Unmatched_Summary.csv
